# Homework 5

## Problem 1

SQP

In [9]:
#pip install qpsolvers[open_source_solvers]

import torch as t

#Initializing x
x = t.tensor([[1.], [1.]]) # x[0] = x1 and x[1] = x2

#Initializing mu and W just to get calculation started -- calculated with initial conditions
u = t.tensor([[6.], [2.]]) # u[0] = u1 and u[1] = u2
W = t.tensor([[2., 0.], [0., (2 + 2*u[0] + 2*u[1])]]) 
w = t.abs(u) #inital weight for line search
alpha = 1 #initial step size

#Calculating Lagrange Gradient
gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  )

#While loop
e = 0.003
while t.norm(gL) > e:
    #Solve for s(k), u(k+1) using QP
    #NOTE: ADD HOW I DID THIS ---------------------------------------------------------------------------------------------------------------------------------------------------- 
    g = t.tensor([[(x[1]**2 - 2*x[0])], [((x[1] - 1)**2 + 5*x[0] -15)]])
    A = t.tensor([[-2., 2*x[1]], [5., (2*x[1] - 2) ]])
    A_inv = t.inverse(A)
    fx = t.tensor([[2*x[0]], [(2*x[1] - 6)]])
    
    s_dum = t.matmul(-t.transpose(g, 0, 1),A_inv)
    s = t.transpose(s_dum, 0, 1) #s(k)
    
    u_first = fx + t.matmul(W, s)
    u_dum = t.matmul(-t.transpose(u_first, 0, 1),A_inv)
    u = t.transpose(u_dum, 0, 1) #mu(k+1)
    
    #Line search, alpha(k), merit function and armijo line search
    
    
    #x(k+1) = x(k) + alpha(k)*s(k)
    x_dum = x + alpha*s
    x = x_dum
    
    #W(k+1), BFGS
    
    
    #Calculate Lagrange Gradient
    gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  ) #need to update x and mu above


KeyboardInterrupt: 

## Problem 2

moon lander